In [31]:
import torch
import torch.nn as nn
import torch.optim as optim

# from chamferdist import ChamferDistance

import open3d as o3d

import numpy as np 
from AE import AE
from PointCloudDataset import PointCloudDataset

In [32]:
path_to_train = "data/shape_net_core_uniform_samples_2048/train.txt"
path_to_val   = "data/shape_net_core_uniform_samples_2048/val.txt"
path_to_test  = "data/shape_net_core_uniform_samples_2048/test.txt"
path_to_tiny  = "data/shape_net_core_uniform_samples_2048/tiny.txt"
path_to_tram  = "data/shape_net_core_uniform_samples_2048/tram.txt"

In [34]:
#Load Train, Val, Test Data

trainset = PointCloudDataset(path_to_data = path_to_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

valset = PointCloudDataset(path_to_data = path_to_val)
valloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = PointCloudDataset(path_to_data = path_to_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True, num_workers=2)

tinyset = PointCloudDataset(path_to_data = path_to_tiny)
tinyloader = torch.utils.data.DataLoader(tinyset, batch_size=5, shuffle=True, num_workers=2)

tramset = PointCloudDataset(path_to_data = path_to_tram)
tramloader = torch.utils.data.DataLoader(tramset, batch_size=5, shuffle=True, num_workers=2)

In [22]:
# batch_size = 128
epochs = 100
learning_rate = 1e-3

In [23]:
testset.data_size

5683

In [24]:
tinyset.data_size

5

In [15]:
tinyset[-1]

(array([ 0.46253878,  0.00937872, -0.03691137, ..., -0.35992891,
         0.00440433,  0.0369372 ]),
 'data/shape_net_core_uniform_samples_2048/04468005/40fcd2ccc96b3fbd041917556492646.ply')

In [18]:
len(trainloader)

315

In [26]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
# model = AE(input_shape=6144).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# mean-squared error loss
criterion = nn.MSELoss()
# criterion = custom_loss
# criterion = ChamferDistance()

In [25]:
model = nn.Sequential(

    nn.Linear(6144, out_features=2048),
    nn.LeakyReLU(),
    nn.BatchNorm1d(2048),
    
    nn.Linear(2048, out_features=1024),
    nn.LeakyReLU(),
    nn.BatchNorm1d(1024),
    
    nn.Linear(1024, out_features=128),
    nn.LeakyReLU(),
    nn.BatchNorm1d(128),
    
    nn.Linear(128, out_features=128),
    nn.LeakyReLU(),
    nn.BatchNorm1d(128),
    
    nn.Linear(128, out_features=128),
    nn.LeakyReLU(),
    nn.BatchNorm1d(128),
   
    nn.Linear(128, out_features=1024),
    nn.LeakyReLU(),
    nn.BatchNorm1d(1024),
    
    nn.Linear(1024, out_features=2048),
    nn.LeakyReLU(),
    nn.BatchNorm1d(2048),
    
    nn.Linear(2048, out_features=6144),
)

In [27]:
for epoch in range(epochs):

    loss = 0
    for index, (batch_features, _) in enumerate(tinyloader):
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device

        batch_features = batch_features.to(device).float()

        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()

        # compute reconstructions
        outputs = model(batch_features).to(device).float()

        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)

        # compute accumulated gradients
        train_loss.backward()

        # perform parameter update based on current gradients
        optimizer.step()

        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()

    # compute the epoch training loss
    loss = loss / len(trainloader)

    # display the epoch training loss
    print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, loss))


epoch : 1/100, recon loss = 0.00116359
epoch : 2/100, recon loss = 0.00112142
epoch : 3/100, recon loss = 0.00098377
epoch : 4/100, recon loss = 0.00086382
epoch : 5/100, recon loss = 0.00068520
epoch : 6/100, recon loss = 0.00055723
epoch : 7/100, recon loss = 0.00041335
epoch : 8/100, recon loss = 0.00034347
epoch : 9/100, recon loss = 0.00030372
epoch : 10/100, recon loss = 0.00025549
epoch : 11/100, recon loss = 0.00022266
epoch : 12/100, recon loss = 0.00020443
epoch : 13/100, recon loss = 0.00018425
epoch : 14/100, recon loss = 0.00016138
epoch : 15/100, recon loss = 0.00014307
epoch : 16/100, recon loss = 0.00013061
epoch : 17/100, recon loss = 0.00012063
epoch : 18/100, recon loss = 0.00011031
epoch : 19/100, recon loss = 0.00009948
epoch : 20/100, recon loss = 0.00008952
epoch : 21/100, recon loss = 0.00008126
epoch : 22/100, recon loss = 0.00007467
epoch : 23/100, recon loss = 0.00006917
epoch : 24/100, recon loss = 0.00006362
epoch : 25/100, recon loss = 0.00005759
epoch : 2

In [35]:
#for predictions
name = "tram_test"
import shutil

for single_test_feature, single_test_path in tramloader:
    input_tensor = single_test_feature.float()         #(128, 6144)
    
    print("input_path:{}".format(single_test_path[0]))
    
    output_tensor = model(input_tensor)                #(128, 6144,)
    reshape_tensor = output_tensor[0].reshape(2048, 3) #(2048, 3)
    #print(output_tensor)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(reshape_tensor.detach().cpu().numpy())
    o3d.io.write_point_cloud("/Users/oswald/Documents/MATLAB/CS231N/{}_predict_ae.ply".format(name), pcd)
    shutil.copyfile(single_test_path[0], "/Users/oswald/Documents/MATLAB/CS231N/{}_original_ae.ply".format(name))
    
    break


input_path:data/shape_net_core_uniform_samples_2048/04468005/19daec6f2602f9c0dc14ba0818ee5cec.ply


In [96]:
import shutil

for single_test_feature, single_test_path in trainloader:
    input_tensor = single_test_feature.float()         #(6144,)
    
    print(single_test_feature.size())
    
    print("input_path:{}".format(single_test_path))
    
#     output_tensor = model(input_tensor)                #(6144,)
    reshape_tensor = input_tensor.reshape(2048, 3)    #(2048, 3)
    print(reshape_tensor.size())
    #print(output_tensor)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(reshape_tensor.detach().cpu().numpy())
    o3d.io.write_point_cloud("./reshaped_input_ae.ply", pcd)
    shutil.copyfile(single_test_path[0], "./original_input_ae.ply")

    break


torch.Size([1, 6144])
input_path:('data/shape_net_core_uniform_samples_2048/02871439/20e2da108eb42b52c16a7f7cb5642902.ply',)
torch.Size([2048, 3])


In [108]:
a = (model.encoder_hidden_layer.weight * 10**9
b = model.encoder_output_layer.weight * 10**9
c = model.decoder_hidden_layer.weight * 10**9
d = model.decoder_output_layer.weight * 100